# Introduction
See https://github.com/wtsi-team112/fits/issues/4

Magnus sent a file with the following files that are in FITS but not in the Pf 6.2 manifest:
```
sample	study_name	irods_path	sanger_sample_id
NULL	Plasmodium falciparum genome variation 1	/seq/6901/6901_8_nonhuman#2.bam	Tr5E
NULL	Plasmodium falciparum genome variation 1	/seq/6901/6901_8_nonhuman#11.bam	Tr3E
NULL	Plasmodium falciparum genome variation 1	/seq/6901/6901_8_nonhuman#10.bam	Tr5E_1
NULL	Plasmodium falciparum genome variation 1	/seq/6901/6901_8_nonhuman#3.bam	Tr3E_1
PD0338-C	Test sequencing of high level human contamination.	/seq/12135/12135_6#66.cram	2876STDY5684799
QE0008-C	Test sequencing of high level human contamination.	/seq/12135/12135_7#68.cram	2876STDY5684801
QE0022-C	Test sequencing of high level human contamination.	/seq/12135/12135_7#70.cram	2876STDY5684803
QE0145-C	Test sequencing of high level human contamination.	/seq/12135/12135_7#69.cram	2876STDY5684802
QE0196-C	Test sequencing of high level human contamination.	/seq/12135/12135_7#67.cram	2876STDY5684800
QE0212-C	Test sequencing of high level human contamination.	/seq/12135/12135_7#71.cram	2876STDY5684804
```
I couldn't see an obvious explanation as to why these weren't in Pf 6.2. Here I investigate these.

In [1]:
%run ../setup.ipynb

python 3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.13.3
scipy 0.19.1
pandas 0.22.0
numexpr 2.6.4
pysam 0.8.4
pysamstats 0.24.3
petlx 1.0.3
vcf 0.6.8
h5py 2.7.0
tables 3.4.2
zarr 2.2.0
scikit-allel 1.1.10


In [2]:
# Inputs
mlwh_missing_exceptions_fn = '/nfs/team112_internal/rp7/src/github/wtsi-team112/SIMS/meta/mlwh/mlwh_missing_exceptions.txt'
mlwh_study_exceptions_fn = '/nfs/team112_internal/rp7/src/github/wtsi-team112/SIMS/meta/mlwh/mlwh_study_exceptions.txt'
mlwh_sample_exceptions_fn = '/nfs/team112_internal/rp7/src/github/wtsi-team112/SIMS/meta/mlwh/mlwh_sample_exceptions.txt'
mlwh_taxon_exceptions_fn = '/nfs/team112_internal/rp7/src/github/wtsi-team112/SIMS/meta/mlwh/mlwh_taxon_exceptions.txt'
sequencescape_alfresco_study_mappings_fn='/nfs/team112_internal/rp7/src/github/wtsi-team112/SIMS/meta/mlwh/sequencescape_alfresco_study_mappings.txt'

# Outputs
output_fn = 'Pf_62_Pv4_like_manifest_20180906.txt'
manifest_symlink = 'pf_62_manifest.txt'
input_dir = '/lustre/scratch118/malaria/team112/pipelines/setups/pf_62/input'

# Does the mlwh query pull in these files?

In [3]:
def set_irods_name(row):
    # First calculate prefix using nonhuman dependent on id_run and species (taxon)
    # Then calculate tag string (empty string if no tag)
    # Then calculate file extension
    # Then concat these three strings
    if (
        (row['taxon_id'] in [7165, 7173, 30066, 62324]) & # Anopheles gambiae, arabiensis, merus and funestus respectively
        (row['id_run'] <= 6750)
    ):
        prefix = "%d_%d_nonhuman" % (row['id_run'], row['position'])
    elif (
        ~(row['taxon_id'] in [7165, 7173, 30066, 62324]) & # Anopheles gambiae, arabiensis, merus and funestus respectively
        (row['id_run'] <= 7100)
    ):
        prefix = "%d_%d_nonhuman" % (row['id_run'], row['position'])
    else:
        prefix = "%d_%d" % (row['id_run'], row['position'])
        
    if np.isnan(row['tag_index']):
        tag_string = ''
    else:
        tag_string = '#%d' % row['tag_index']
    
    if row['id_run'] <= 10300:
        file_extension = '.bam'
    else:
        file_extension = '.cram'
    
    irods_filename = prefix + tag_string + file_extension
    return(irods_filename)

def set_sample_id(row):
    if row['sample'] is None:
        return('DS_%s' % row['name'])
    else:
        return('DS_%s' % row['sample'])

def is_valid_alfresco_code(s):
    try: 
        int(s[0:4])
        return True
    except ValueError:
        return False

def create_build_manifest(
#     taxon_ids=[5833, 5855, 5858, 5821, 7165, 7173, 30066], # Pf, Pv, Pm, Pb, Ag, A. arabiensis, A. merus
    taxon_ids=[5833, 36329, 5847, 57267, 137071], # Pf, 3D7, V1, Dd2, HB3
    # 1089 is excluded as this is R&D samples. 1204 and 1176 are excluded as these are CP1 samples. 1175 is excluded as some Pf R&D samples were incorrectly tagged with this study
    studies_to_exclude = ['1089-R&amp;D-GB-ALCOCK', '1204-PF-GM-CP1', '1176-PF-KE-CP1', '1175-VO-KH-STLAURENT'],
    miseq_runs_to_allow = [13809, 13810], # Two Miseq runs within 24 samples on each from study 1095-PF-TZ-ISHENGOMA that were included in Pf 6.0
    input_dir = input_dir,
    mlwh_missing_exceptions_fn = mlwh_missing_exceptions_fn,
    mlwh_study_exceptions_fn = mlwh_study_exceptions_fn,
    mlwh_sample_exceptions_fn = mlwh_sample_exceptions_fn,
    mlwh_taxon_exceptions_fn = mlwh_taxon_exceptions_fn,
    sequencescape_alfresco_study_mappings_fn = sequencescape_alfresco_study_mappings_fn,
    output_columns = [
        'path',
        'study',
        'sample',
        'lane',
        'reads',
        'paired',
        'irods_path',
        'sanger_sample_id',
        'taxon_id',
        'study_lims',
        'study_name',
        'id_run',
        'position',
        'tag_index',
        'qc_complete',
        'manual_qc',
        'description',
        'instrument_name',
        'instrument_model',
        'forward_read_length',
        'requested_insert_size_from',
        'requested_insert_size_to',
        'human_percent_mapped',
        'subtrack_filename',
        'subtrack_files_bytes',
        'ebi_run_acc'
    ]
):
    """Create a DataFrame to be used as a build manifest.
    
    A build manifest here is a list of all the "lanelets" that need to be
    included in a build. The output will typically be written to a
    tab-delmited file, either to use as input to vr-pipe, or perhaps as
    input to a vr-track DB.

    Args:
        taxon_ids (list of int): The taxons of the build (P. falciparum is '5833',
            P. vivax is '5855').
        sequencescape_alfresco_study_mappings_fn (str): filename of
            mappings from sequencscape to alfresco study mappings

    Returns:
        pd.DataFrame: The build manifest.

    """
    
    # Read in taxon exceptions file
    df_mlwh_taxon_exceptions = pd.read_csv(mlwh_taxon_exceptions_fn, sep='\t', dtype={'tag_index': str, 'taxon_id': int}, index_col='irods_filename')
    df_mlwh_taxon_exceptions['tag_index'].fillna('', inplace=True)
    # Identify which samples in exceptions file match the taxon of current build, and which don't
    df_mlwh_exceptions_this_taxon = df_mlwh_taxon_exceptions.loc[df_mlwh_taxon_exceptions['taxon_id'].isin(taxon_ids)]
    df_mlwh_exceptions_other_taxa = df_mlwh_taxon_exceptions.loc[~(df_mlwh_taxon_exceptions['taxon_id'].isin(taxon_ids))]

    # Read in sample exceptions file
    df_mlwh_sample_exceptions = pd.read_csv(mlwh_sample_exceptions_fn, sep='\t', index_col='irods_filename')

    # Read in study exceptions file
    df_mlwh_study_exceptions = pd.read_csv(mlwh_study_exceptions_fn, sep='\t', index_col='irods_filename')

    # Read in missing exceptions file
    df_mlwh_missing_exceptions = pd.read_csv(mlwh_missing_exceptions_fn, sep='\t', index_col='irods_filename')
    df_mlwh_missing_exceptions['derivative_sample_id'] = 'DS_' + df_mlwh_missing_exceptions['sample_id']
    df_mlwh_missing_exceptions = df_mlwh_missing_exceptions.loc[
        df_mlwh_missing_exceptions['taxon_id'].isin(taxon_ids)
    ]

    # Read in SequenceScape-Alfresco study mappings
    df_sequencescape_alfresco_study_mappings = pd.read_csv(sequencescape_alfresco_study_mappings_fn, sep='\t', index_col='seqscape_study_id')
    sequencescape_alfresco_study_mappings_dict = df_sequencescape_alfresco_study_mappings.loc[
        :,
#         df_sequencescape_alfresco_study_mappings['build_flag']==1,
        'alfresco_study_code'
    ].to_dict()
    
    # Read in data from mlwh matching this taxon, plus samples from exceptions file matching this taxon
    conn = pymysql.connect(
        host='mlwh-db',
        user='mlwh_malaria',
        password='Solaris&2015',
        db='mlwarehouse',
        port=3435
    )
    
    sql_query = 'SELECT \
        study.name as study_name, \
        study.id_study_lims as study_lims, \
        sample.supplier_name as sample, \
        sample.name, \
        sample.sanger_sample_id, \
        sample.taxon_id, \
        iseq_product_metrics.id_run, \
        iseq_product_metrics.position, \
        iseq_product_metrics.tag_index, \
        iseq_product_metrics.num_reads, \
        iseq_product_metrics.human_percent_mapped, \
        iseq_run_lane_metrics.instrument_name, \
        iseq_run_lane_metrics.instrument_model, \
        iseq_run_lane_metrics.paired_read, \
        iseq_run_lane_metrics.qc_complete, \
        iseq_flowcell.manual_qc, \
        iseq_flowcell.requested_insert_size_from, \
        iseq_flowcell.requested_insert_size_to, \
        iseq_flowcell.forward_read_length, \
        iseq_run_status_dict.description \
    FROM \
        study, \
        iseq_flowcell, \
        sample, \
        iseq_product_metrics, \
        iseq_run_status, \
        iseq_run_lane_metrics, \
        iseq_run_status_dict \
    WHERE \
        study.id_study_tmp = iseq_flowcell.id_study_tmp and \
        iseq_flowcell.id_sample_tmp = sample.id_sample_tmp and \
        iseq_flowcell.manual_qc = 1 and \
        iseq_product_metrics.id_iseq_flowcell_tmp = iseq_flowcell.id_iseq_flowcell_tmp and \
        iseq_run_status.id_run = iseq_product_metrics.id_run and \
        iseq_product_metrics.id_run = iseq_run_lane_metrics.id_run and \
        iseq_product_metrics.position = iseq_run_lane_metrics.position and \
        iseq_run_status.iscurrent = 1 and \
        ( ( iseq_run_lane_metrics.instrument_model != "MiSeq" ) or (iseq_product_metrics.id_run in (%s)) ) and \
        iseq_run_status.id_run_status_dict = iseq_run_status_dict.id_run_status_dict and \
        study.faculty_sponsor = "Dominic Kwiatkowski" and \
        ( ( sample.taxon_id in (%s) ) or ' % (
            ', '.join([str(x) for x in miseq_runs_to_allow]),
            ', '.join([str(x) for x in taxon_ids])
        )
    sql_query = sql_query + ' or '.join(
        df_mlwh_taxon_exceptions.loc[df_mlwh_taxon_exceptions['taxon_id'].isin(taxon_ids)].apply(
            lambda x: '(iseq_product_metrics.id_run="%s" and iseq_product_metrics.position="%s" and iseq_product_metrics.tag_index="%s")' % (x['id_run'], x['position'], x['tag_index']),
            1
        )
    )
    sql_query = sql_query + ')'
#         (iseq_flowcell.manual_qc = 1 or iseq_flowcell.manual_qc is null) and \
    
    df_return = pd.read_sql(sql_query, conn)

    # Replace missing taxon_id with -1 (can't have missing int values in pandas Series)
    df_return['taxon_id'] = df_return['taxon_id'].fillna(-1).astype('int32')

    # Determine file name in iRods
    df_return['irods_filename'] = df_return.apply(set_irods_name, 1)
    df_return.set_index('irods_filename', inplace=True)

    return(df_return)



In [4]:
df_mlwh = create_build_manifest()
df_mlwh[0:3]

,study_name,study_lims,sample,name,sanger_sample_id,taxon_id,id_run,position,tag_index,num_reads,human_percent_mapped,instrument_name,instrument_model,paired_read,qc_complete,manual_qc,requested_insert_size_from,requested_insert_size_to,forward_read_length,description
irods_filename,,,,,,,,,,,,,,,,,,,,
1337_1_nonhuman.bam,Plasmodium falciparum genome variation 1,578,None,AK121_D80 300,None,5833,1337,1,NaN,2745194,60.35,IL29,HK,1,2008-09-24 15:54:04,1,300.0,300.0,37,qc complete
1337_2_nonhuman.bam,Plasmodium falciparum genome variation 1,578,None,AK121_D80 200,None,5833,1337,2,NaN,4310986,58.68,IL29,HK,1,2008-09-24 15:54:04,1,200.0,200.0,37,qc complete
1337_3_nonhuman.bam,Plasmodium falciparum genome variation 1,578,None,AK121_D80 200,None,5833,1337,3,NaN,14488586,57.19,IL29,HK,1,2008-09-24 15:54:04,1,200.0,200.0,37,qc complete


In [5]:
df_mlwh.loc[df_mlwh['id_run'] == 6901]

,study_name,study_lims,sample,name,sanger_sample_id,taxon_id,id_run,position,tag_index,num_reads,human_percent_mapped,instrument_name,instrument_model,paired_read,qc_complete,manual_qc,requested_insert_size_from,requested_insert_size_to,forward_read_length,description
irods_filename,,,,,,,,,,,,,,,,,,,,
6901_3_nonhuman#1.bam,Plasmodium falciparum genome variation 1,578,None,PF0477-C,None,5833,6901,3,1.0,15195722,89.33,HS13,HiSeq,1,2011-11-01 12:52:25,1,200.0,300.0,100,qc complete
6901_3_nonhuman#2.bam,Plasmodium falciparum genome variation 1,578,None,PF0479-C,None,5833,6901,3,2.0,22994194,90.78,HS13,HiSeq,1,2011-11-01 12:52:25,1,200.0,300.0,100,qc complete
6901_3_nonhuman#3.bam,Plasmodium falciparum genome variation 1,578,None,PF0473-C,None,5833,6901,3,3.0,18145404,80.67,HS13,HiSeq,1,2011-11-01 12:52:25,1,200.0,300.0,100,qc complete
6901_3_nonhuman#4.bam,Plasmodium falciparum genome variation 1,578,None,PF0522-C,None,5833,6901,3,4.0,22990098,74.86,HS13,HiSeq,1,2011-11-01 12:52:25,1,200.0,300.0,100,qc complete
6901_3_nonhuman#5.bam,Plasmodium falciparum genome variation 1,578,None,PF0463-C,None,5833,6901,3,5.0,15795786,85.70,HS13,HiSeq,1,2011-11-01 12:52:25,1,200.0,300.0,100,qc complete
6901_3_nonhuman#6.bam,Plasmodium falciparum genome variation 1,578,None,PF0531-C,None,5833,6901,3,6.0,16618420,81.78,HS13,HiSeq,1,2011-11-01 12:52:25,1,200.0,300.0,100,qc complete
6901_3_nonhuman#7.bam,Plasmodium falciparum genome variation 1,578,None,PF0512-C,None,5833,6901,3,7.0,20197362,87.67,HS13,HiSeq,1,2011-11-01 12:52:25,1,200.0,300.0,100,qc complete
6901_3_nonhuman#8.bam,Plasmodium falciparum genome variation 1,578,None,PF0454-C,None,5833,6901,3,8.0,19465296,91.49,HS13,HiSeq,1,2011-11-01 12:52:25,1,200.0,300.0,100,qc complete
6901_3_nonhuman#9.bam,Plasmodium falciparum genome variation 1,578,None,PF0472-C,None,5833,6901,3,9.0,14194180,96.48,HS13,HiSeq,1,2011-11-01 12:52:25,1,200.0,300.0,100,qc complete


In [6]:
'6901_8_nonhuman#1.bam' in df_mlwh.index

True

In [7]:
'6901_8_nonhuman#2.bam' in df_mlwh.index

True

In [8]:
df_mlwh.loc[df_mlwh['id_run'] == 12135]

,study_name,study_lims,sample,name,sanger_sample_id,taxon_id,id_run,position,tag_index,num_reads,human_percent_mapped,instrument_name,instrument_model,paired_read,qc_complete,manual_qc,requested_insert_size_from,requested_insert_size_to,forward_read_length,description
irods_filename,,,,,,,,,,,,,,,,,,,,
12135_1#31.cram,Test sequencing of high level human contaminat...,2876,PC0138-C,2876STDY5684764,2876STDY5684764,5833,12135,1,31.0,2512866,99.55,HS16,HiSeq,1,2014-02-28 12:53:57,1,200.0,300.0,100,qc complete
12135_1#32.cram,Test sequencing of high level human contaminat...,2876,PC0139-C,2876STDY5684765,2876STDY5684765,5833,12135,1,32.0,2193340,99.22,HS16,HiSeq,1,2014-02-28 12:53:57,1,200.0,300.0,100,qc complete
12135_1#33.cram,Test sequencing of high level human contaminat...,2876,PC0140-C,2876STDY5684766,2876STDY5684766,5833,12135,1,33.0,2098154,99.49,HS16,HiSeq,1,2014-02-28 12:53:57,1,200.0,300.0,100,qc complete
12135_1#34.cram,Test sequencing of high level human contaminat...,2876,PC0141-C,2876STDY5684767,2876STDY5684767,5833,12135,1,34.0,2209976,99.53,HS16,HiSeq,1,2014-02-28 12:53:57,1,200.0,300.0,100,qc complete
12135_1#35.cram,Test sequencing of high level human contaminat...,2876,PC0142-C,2876STDY5684768,2876STDY5684768,5833,12135,1,35.0,3432242,99.52,HS16,HiSeq,1,2014-02-28 12:53:57,1,200.0,300.0,100,qc complete
12135_1#36.cram,Test sequencing of high level human contaminat...,2876,PC0143-C,2876STDY5684769,2876STDY5684769,5833,12135,1,36.0,2457978,99.52,HS16,HiSeq,1,2014-02-28 12:53:57,1,200.0,300.0,100,qc complete
12135_2#37.cram,Test sequencing of high level human contaminat...,2876,PC0144-C,2876STDY5684770,2876STDY5684770,5833,12135,2,37.0,2077158,99.52,HS16,HiSeq,1,2014-02-28 12:53:57,1,200.0,300.0,100,qc complete
12135_2#38.cram,Test sequencing of high level human contaminat...,2876,PC0145-C,2876STDY5684771,2876STDY5684771,5833,12135,2,38.0,2133750,99.49,HS16,HiSeq,1,2014-02-28 12:53:57,1,200.0,300.0,100,qc complete
12135_2#39.cram,Test sequencing of high level human contaminat...,2876,PC0146-C,2876STDY5684772,2876STDY5684772,5833,12135,2,39.0,2447318,99.53,HS16,HiSeq,1,2014-02-28 12:53:57,1,200.0,300.0,100,qc complete


In [9]:
'12135_6#65.cram' in df_mlwh.index

True

In [10]:
'12135_6#66.cram' in df_mlwh.index

True

# Add some constraints

In [11]:
def set_irods_name(row):
    # First calculate prefix using nonhuman dependent on id_run and species (taxon)
    # Then calculate tag string (empty string if no tag)
    # Then calculate file extension
    # Then concat these three strings
    if (
        (row['taxon_id'] in [7165, 7173, 30066, 62324]) & # Anopheles gambiae, arabiensis, merus and funestus respectively
        (row['id_run'] <= 6750)
    ):
        prefix = "%d_%d_nonhuman" % (row['id_run'], row['position'])
    elif (
        ~(row['taxon_id'] in [7165, 7173, 30066, 62324]) & # Anopheles gambiae, arabiensis, merus and funestus respectively
        (row['id_run'] <= 7100)
    ):
        prefix = "%d_%d_nonhuman" % (row['id_run'], row['position'])
    else:
        prefix = "%d_%d" % (row['id_run'], row['position'])
        
    if np.isnan(row['tag_index']):
        tag_string = ''
    else:
        tag_string = '#%d' % row['tag_index']
    
    if row['id_run'] <= 10300:
        file_extension = '.bam'
    else:
        file_extension = '.cram'
    
    irods_filename = prefix + tag_string + file_extension
    return(irods_filename)

def set_sample_id(row):
    if row['sample'] is None:
        return('DS_%s' % row['name'])
    else:
        return('DS_%s' % row['sample'])

def is_valid_alfresco_code(s):
    try: 
        int(s[0:4])
        return True
    except ValueError:
        return False

def create_build_manifest(
#     taxon_ids=[5833, 5855, 5858, 5821, 7165, 7173, 30066], # Pf, Pv, Pm, Pb, Ag, A. arabiensis, A. merus
    taxon_ids=[5833, 36329, 5847, 57267, 137071], # Pf, 3D7, V1, Dd2, HB3
    # 1089 is excluded as this is R&D samples. 1204 and 1176 are excluded as these are CP1 samples. 1175 is excluded as some Pf R&D samples were incorrectly tagged with this study
    studies_to_exclude = ['1089-R&amp;D-GB-ALCOCK', '1204-PF-GM-CP1', '1176-PF-KE-CP1', '1175-VO-KH-STLAURENT'],
    miseq_runs_to_allow = [13809, 13810], # Two Miseq runs within 24 samples on each from study 1095-PF-TZ-ISHENGOMA that were included in Pf 6.0
    input_dir = input_dir,
    mlwh_missing_exceptions_fn = mlwh_missing_exceptions_fn,
    mlwh_study_exceptions_fn = mlwh_study_exceptions_fn,
    mlwh_sample_exceptions_fn = mlwh_sample_exceptions_fn,
    mlwh_taxon_exceptions_fn = mlwh_taxon_exceptions_fn,
    sequencescape_alfresco_study_mappings_fn = sequencescape_alfresco_study_mappings_fn,
    output_columns = [
        'path',
        'study',
        'sample',
        'lane',
        'reads',
        'paired',
        'irods_path',
        'sanger_sample_id',
        'taxon_id',
        'study_lims',
        'study_name',
        'id_run',
        'position',
        'tag_index',
        'qc_complete',
        'manual_qc',
        'description',
        'instrument_name',
        'instrument_model',
        'forward_read_length',
        'requested_insert_size_from',
        'requested_insert_size_to',
        'human_percent_mapped',
        'subtrack_filename',
        'subtrack_files_bytes',
        'ebi_run_acc'
    ]
):
    """Create a DataFrame to be used as a build manifest.
    
    A build manifest here is a list of all the "lanelets" that need to be
    included in a build. The output will typically be written to a
    tab-delmited file, either to use as input to vr-pipe, or perhaps as
    input to a vr-track DB.

    Args:
        taxon_ids (list of int): The taxons of the build (P. falciparum is '5833',
            P. vivax is '5855').
        sequencescape_alfresco_study_mappings_fn (str): filename of
            mappings from sequencscape to alfresco study mappings

    Returns:
        pd.DataFrame: The build manifest.

    """
    
    # Read in taxon exceptions file
    df_mlwh_taxon_exceptions = pd.read_csv(mlwh_taxon_exceptions_fn, sep='\t', dtype={'tag_index': str, 'taxon_id': int}, index_col='irods_filename')
    df_mlwh_taxon_exceptions['tag_index'].fillna('', inplace=True)
    # Identify which samples in exceptions file match the taxon of current build, and which don't
    df_mlwh_exceptions_this_taxon = df_mlwh_taxon_exceptions.loc[df_mlwh_taxon_exceptions['taxon_id'].isin(taxon_ids)]
    df_mlwh_exceptions_other_taxa = df_mlwh_taxon_exceptions.loc[~(df_mlwh_taxon_exceptions['taxon_id'].isin(taxon_ids))]

    # Read in sample exceptions file
    df_mlwh_sample_exceptions = pd.read_csv(mlwh_sample_exceptions_fn, sep='\t', index_col='irods_filename')

    # Read in study exceptions file
    df_mlwh_study_exceptions = pd.read_csv(mlwh_study_exceptions_fn, sep='\t', index_col='irods_filename')

    # Read in missing exceptions file
    df_mlwh_missing_exceptions = pd.read_csv(mlwh_missing_exceptions_fn, sep='\t', index_col='irods_filename')
    df_mlwh_missing_exceptions['derivative_sample_id'] = 'DS_' + df_mlwh_missing_exceptions['sample_id']
    df_mlwh_missing_exceptions = df_mlwh_missing_exceptions.loc[
        df_mlwh_missing_exceptions['taxon_id'].isin(taxon_ids)
    ]

    # Read in SequenceScape-Alfresco study mappings
    df_sequencescape_alfresco_study_mappings = pd.read_csv(sequencescape_alfresco_study_mappings_fn, sep='\t', index_col='seqscape_study_id')
    sequencescape_alfresco_study_mappings_dict = df_sequencescape_alfresco_study_mappings.loc[
        :,
#         df_sequencescape_alfresco_study_mappings['build_flag']==1,
        'alfresco_study_code'
    ].to_dict()
    
    # Read in data from mlwh matching this taxon, plus samples from exceptions file matching this taxon
    conn = pymysql.connect(
        host='mlwh-db',
        user='mlwh_malaria',
        password='Solaris&2015',
        db='mlwarehouse',
        port=3435
    )
    
    sql_query = 'SELECT \
        study.name as study_name, \
        study.id_study_lims as study_lims, \
        sample.supplier_name as sample, \
        sample.name, \
        sample.sanger_sample_id, \
        sample.taxon_id, \
        iseq_product_metrics.id_run, \
        iseq_product_metrics.position, \
        iseq_product_metrics.tag_index, \
        iseq_product_metrics.num_reads, \
        iseq_product_metrics.human_percent_mapped, \
        iseq_run_lane_metrics.instrument_name, \
        iseq_run_lane_metrics.instrument_model, \
        iseq_run_lane_metrics.paired_read, \
        iseq_run_lane_metrics.qc_complete, \
        iseq_flowcell.manual_qc, \
        iseq_flowcell.requested_insert_size_from, \
        iseq_flowcell.requested_insert_size_to, \
        iseq_flowcell.forward_read_length, \
        iseq_run_status_dict.description \
    FROM \
        study, \
        iseq_flowcell, \
        sample, \
        iseq_product_metrics, \
        iseq_run_status, \
        iseq_run_lane_metrics, \
        iseq_run_status_dict \
    WHERE \
        study.id_study_tmp = iseq_flowcell.id_study_tmp and \
        iseq_flowcell.id_sample_tmp = sample.id_sample_tmp and \
        iseq_flowcell.manual_qc = 1 and \
        iseq_product_metrics.id_iseq_flowcell_tmp = iseq_flowcell.id_iseq_flowcell_tmp and \
        iseq_run_status.id_run = iseq_product_metrics.id_run and \
        iseq_product_metrics.id_run = iseq_run_lane_metrics.id_run and \
        iseq_product_metrics.position = iseq_run_lane_metrics.position and \
        iseq_run_status.iscurrent = 1 and \
        ( ( iseq_run_lane_metrics.instrument_model != "MiSeq" ) or (iseq_product_metrics.id_run in (%s)) ) and \
        iseq_run_status.id_run_status_dict = iseq_run_status_dict.id_run_status_dict and \
        study.faculty_sponsor = "Dominic Kwiatkowski" and \
        ( ( sample.taxon_id in (%s) ) or ' % (
            ', '.join([str(x) for x in miseq_runs_to_allow]),
            ', '.join([str(x) for x in taxon_ids])
        )
    sql_query = sql_query + ' or '.join(
        df_mlwh_taxon_exceptions.loc[df_mlwh_taxon_exceptions['taxon_id'].isin(taxon_ids)].apply(
            lambda x: '(iseq_product_metrics.id_run="%s" and iseq_product_metrics.position="%s" and iseq_product_metrics.tag_index="%s")' % (x['id_run'], x['position'], x['tag_index']),
            1
        )
    )
    sql_query = sql_query + ')'
#         (iseq_flowcell.manual_qc = 1 or iseq_flowcell.manual_qc is null) and \
    
    df_return = pd.read_sql(sql_query, conn)

    # Replace missing taxon_id with -1 (can't have missing int values in pandas Series)
    df_return['taxon_id'] = df_return['taxon_id'].fillna(-1).astype('int32')

    # Determine file name in iRods
    df_return['irods_filename'] = df_return.apply(set_irods_name, 1)
    df_return.set_index('irods_filename', inplace=True)

    # Determine alfresco study and change any incorrect studies
    df_return['alfresco_study_code'] = df_return['study_lims'].apply(
        lambda x: sequencescape_alfresco_study_mappings_dict[int(x)] if int(x) in sequencescape_alfresco_study_mappings_dict else ''
    )
    empty_alfresco_study_code = (df_return['alfresco_study_code'] == '')
    df_return.loc[empty_alfresco_study_code, 'alfresco_study_code'] = df_return.loc[empty_alfresco_study_code, 'study_name']
    
    study_exception_indexes = df_mlwh_study_exceptions.index[
        df_mlwh_study_exceptions.index.isin(df_return.index)
    ]
    df_return.loc[study_exception_indexes, 'alfresco_study_code'] = df_mlwh_study_exceptions.loc[study_exception_indexes, 'alfresco_study_code']

    # Change any incorrect taxon IDs
    taxon_exception_indexes = df_mlwh_exceptions_this_taxon.index[
        df_mlwh_exceptions_this_taxon.index.to_series().isin(df_return.index)
    ]
    df_return.loc[taxon_exception_indexes, 'taxon_id'] = df_mlwh_exceptions_this_taxon.loc[taxon_exception_indexes, 'taxon_id']

    # Determine sample ID and change any incorrect sample IDs
    df_return['derivative_sample_id'] = df_return.apply(set_sample_id, 1)
    sample_exception_indexes = df_mlwh_sample_exceptions.index[
        df_mlwh_sample_exceptions.index.isin(df_return.index)
    ]    
    df_return.loc[sample_exception_indexes, 'derivative_sample_id'] = 'DS_' + df_mlwh_sample_exceptions.loc[sample_exception_indexes, 'sample_id']
    df_return.drop(['sample', 'name'], axis=1, inplace=True)
    
    # Remove any samples that are not in this taxon
    df_return = df_return.loc[~df_return.index.isin(df_mlwh_exceptions_other_taxa.index)]

    return(df_return)



In [12]:
df_mlwh_taxa_exceptions = create_build_manifest()
print(df_mlwh_taxa_exceptions.shape)
print('6901_8_nonhuman#1.bam' in df_mlwh_taxa_exceptions.index)
print('6901_8_nonhuman#2.bam' in df_mlwh_taxa_exceptions.index)
print('12135_6#65.cram' in df_mlwh_taxa_exceptions.index)
print('12135_6#66.cram' in df_mlwh_taxa_exceptions.index)


(23430, 20)
True
True
True
True


In [13]:
df_mlwh_taxa_exceptions.loc[['6901_8_nonhuman#1.bam', '6901_8_nonhuman#2.bam']]

,study_name,study_lims,sanger_sample_id,taxon_id,id_run,position,tag_index,num_reads,human_percent_mapped,instrument_name,instrument_model,paired_read,qc_complete,manual_qc,requested_insert_size_from,requested_insert_size_to,forward_read_length,description,alfresco_study_code,derivative_sample_id
irods_filename,,,,,,,,,,,,,,,,,,,,
6901_8_nonhuman#1.bam,Plasmodium falciparum genome variation 1,578,None,5833,6901,8,1.0,4969844,99.42,HS13,HiSeq,1,2011-11-01 12:52:25,1,200.0,300.0,100,qc complete,1052-PF-TRAC-WHITE,DS_PH0540-C
6901_8_nonhuman#2.bam,Plasmodium falciparum genome variation 1,578,None,5833,6901,8,2.0,52856,93.42,HS13,HiSeq,1,2011-11-01 12:52:25,1,200.0,300.0,100,qc complete,Plasmodium falciparum genome variation 1,DS_Tr5E


### Summary
Missing files still included here - add more constraints...

In [14]:
def create_build_manifest(
#     taxon_ids=[5833, 5855, 5858, 5821, 7165, 7173, 30066], # Pf, Pv, Pm, Pb, Ag, A. arabiensis, A. merus
    taxon_ids=[5833, 36329, 5847, 57267, 137071], # Pf, 3D7, V1, Dd2, HB3
    # 1089 is excluded as this is R&D samples. 1204 and 1176 are excluded as these are CP1 samples. 1175 is excluded as some Pf R&D samples were incorrectly tagged with this study
    studies_to_exclude = ['1089-R&amp;D-GB-ALCOCK', '1204-PF-GM-CP1', '1176-PF-KE-CP1', '1175-VO-KH-STLAURENT'],
    miseq_runs_to_allow = [13809, 13810], # Two Miseq runs within 24 samples on each from study 1095-PF-TZ-ISHENGOMA that were included in Pf 6.0
    input_dir = input_dir,
    mlwh_missing_exceptions_fn = mlwh_missing_exceptions_fn,
    mlwh_study_exceptions_fn = mlwh_study_exceptions_fn,
    mlwh_sample_exceptions_fn = mlwh_sample_exceptions_fn,
    mlwh_taxon_exceptions_fn = mlwh_taxon_exceptions_fn,
    sequencescape_alfresco_study_mappings_fn = sequencescape_alfresco_study_mappings_fn,
    output_columns = [
        'path',
        'study',
        'sample',
        'lane',
        'reads',
        'paired',
        'irods_path',
        'sanger_sample_id',
        'taxon_id',
        'study_lims',
        'study_name',
        'id_run',
        'position',
        'tag_index',
        'qc_complete',
        'manual_qc',
        'description',
        'instrument_name',
        'instrument_model',
        'forward_read_length',
        'requested_insert_size_from',
        'requested_insert_size_to',
        'human_percent_mapped',
        'subtrack_filename',
        'subtrack_files_bytes',
        'ebi_run_acc'
    ]
):
    """Create a DataFrame to be used as a build manifest.
    
    A build manifest here is a list of all the "lanelets" that need to be
    included in a build. The output will typically be written to a
    tab-delmited file, either to use as input to vr-pipe, or perhaps as
    input to a vr-track DB.

    Args:
        taxon_ids (list of int): The taxons of the build (P. falciparum is '5833',
            P. vivax is '5855').
        sequencescape_alfresco_study_mappings_fn (str): filename of
            mappings from sequencscape to alfresco study mappings

    Returns:
        pd.DataFrame: The build manifest.

    """
    
    # Read in taxon exceptions file
    df_mlwh_taxon_exceptions = pd.read_csv(mlwh_taxon_exceptions_fn, sep='\t', dtype={'tag_index': str, 'taxon_id': int}, index_col='irods_filename')
    df_mlwh_taxon_exceptions['tag_index'].fillna('', inplace=True)
    # Identify which samples in exceptions file match the taxon of current build, and which don't
    df_mlwh_exceptions_this_taxon = df_mlwh_taxon_exceptions.loc[df_mlwh_taxon_exceptions['taxon_id'].isin(taxon_ids)]
    df_mlwh_exceptions_other_taxa = df_mlwh_taxon_exceptions.loc[~(df_mlwh_taxon_exceptions['taxon_id'].isin(taxon_ids))]

    # Read in sample exceptions file
    df_mlwh_sample_exceptions = pd.read_csv(mlwh_sample_exceptions_fn, sep='\t', index_col='irods_filename')

    # Read in study exceptions file
    df_mlwh_study_exceptions = pd.read_csv(mlwh_study_exceptions_fn, sep='\t', index_col='irods_filename')

    # Read in missing exceptions file
    df_mlwh_missing_exceptions = pd.read_csv(mlwh_missing_exceptions_fn, sep='\t', index_col='irods_filename')
    df_mlwh_missing_exceptions['derivative_sample_id'] = 'DS_' + df_mlwh_missing_exceptions['sample_id']
    df_mlwh_missing_exceptions = df_mlwh_missing_exceptions.loc[
        df_mlwh_missing_exceptions['taxon_id'].isin(taxon_ids)
    ]

    # Read in SequenceScape-Alfresco study mappings
    df_sequencescape_alfresco_study_mappings = pd.read_csv(sequencescape_alfresco_study_mappings_fn, sep='\t', index_col='seqscape_study_id')
    sequencescape_alfresco_study_mappings_dict = df_sequencescape_alfresco_study_mappings.loc[
        :,
#         df_sequencescape_alfresco_study_mappings['build_flag']==1,
        'alfresco_study_code'
    ].to_dict()
    
    # Read in data from mlwh matching this taxon, plus samples from exceptions file matching this taxon
    conn = pymysql.connect(
        host='mlwh-db',
        user='mlwh_malaria',
        password='Solaris&2015',
        db='mlwarehouse',
        port=3435
    )
    
    sql_query = 'SELECT \
        study.name as study_name, \
        study.id_study_lims as study_lims, \
        sample.supplier_name as sample, \
        sample.name, \
        sample.sanger_sample_id, \
        sample.taxon_id, \
        iseq_product_metrics.id_run, \
        iseq_product_metrics.position, \
        iseq_product_metrics.tag_index, \
        iseq_product_metrics.num_reads, \
        iseq_product_metrics.human_percent_mapped, \
        iseq_run_lane_metrics.instrument_name, \
        iseq_run_lane_metrics.instrument_model, \
        iseq_run_lane_metrics.paired_read, \
        iseq_run_lane_metrics.qc_complete, \
        iseq_flowcell.manual_qc, \
        iseq_flowcell.requested_insert_size_from, \
        iseq_flowcell.requested_insert_size_to, \
        iseq_flowcell.forward_read_length, \
        iseq_run_status_dict.description \
    FROM \
        study, \
        iseq_flowcell, \
        sample, \
        iseq_product_metrics, \
        iseq_run_status, \
        iseq_run_lane_metrics, \
        iseq_run_status_dict \
    WHERE \
        study.id_study_tmp = iseq_flowcell.id_study_tmp and \
        iseq_flowcell.id_sample_tmp = sample.id_sample_tmp and \
        iseq_flowcell.manual_qc = 1 and \
        iseq_product_metrics.id_iseq_flowcell_tmp = iseq_flowcell.id_iseq_flowcell_tmp and \
        iseq_run_status.id_run = iseq_product_metrics.id_run and \
        iseq_product_metrics.id_run = iseq_run_lane_metrics.id_run and \
        iseq_product_metrics.position = iseq_run_lane_metrics.position and \
        iseq_run_status.iscurrent = 1 and \
        ( ( iseq_run_lane_metrics.instrument_model != "MiSeq" ) or (iseq_product_metrics.id_run in (%s)) ) and \
        iseq_run_status.id_run_status_dict = iseq_run_status_dict.id_run_status_dict and \
        study.faculty_sponsor = "Dominic Kwiatkowski" and \
        ( ( sample.taxon_id in (%s) ) or ' % (
            ', '.join([str(x) for x in miseq_runs_to_allow]),
            ', '.join([str(x) for x in taxon_ids])
        )
    sql_query = sql_query + ' or '.join(
        df_mlwh_taxon_exceptions.loc[df_mlwh_taxon_exceptions['taxon_id'].isin(taxon_ids)].apply(
            lambda x: '(iseq_product_metrics.id_run="%s" and iseq_product_metrics.position="%s" and iseq_product_metrics.tag_index="%s")' % (x['id_run'], x['position'], x['tag_index']),
            1
        )
    )
    sql_query = sql_query + ')'
#         (iseq_flowcell.manual_qc = 1 or iseq_flowcell.manual_qc is null) and \
    
    df_return = pd.read_sql(sql_query, conn)

    # Replace missing taxon_id with -1 (can't have missing int values in pandas Series)
    df_return['taxon_id'] = df_return['taxon_id'].fillna(-1).astype('int32')

    # Determine file name in iRods
    df_return['irods_filename'] = df_return.apply(set_irods_name, 1)
    df_return.set_index('irods_filename', inplace=True)

    # Determine alfresco study and change any incorrect studies
    df_return['alfresco_study_code'] = df_return['study_lims'].apply(
        lambda x: sequencescape_alfresco_study_mappings_dict[int(x)] if int(x) in sequencescape_alfresco_study_mappings_dict else ''
    )
    empty_alfresco_study_code = (df_return['alfresco_study_code'] == '')
    df_return.loc[empty_alfresco_study_code, 'alfresco_study_code'] = df_return.loc[empty_alfresco_study_code, 'study_name']
    
    study_exception_indexes = df_mlwh_study_exceptions.index[
        df_mlwh_study_exceptions.index.isin(df_return.index)
    ]
    df_return.loc[study_exception_indexes, 'alfresco_study_code'] = df_mlwh_study_exceptions.loc[study_exception_indexes, 'alfresco_study_code']

    # Change any incorrect taxon IDs
    taxon_exception_indexes = df_mlwh_exceptions_this_taxon.index[
        df_mlwh_exceptions_this_taxon.index.to_series().isin(df_return.index)
    ]
    df_return.loc[taxon_exception_indexes, 'taxon_id'] = df_mlwh_exceptions_this_taxon.loc[taxon_exception_indexes, 'taxon_id']

    # Determine sample ID and change any incorrect sample IDs
    df_return['derivative_sample_id'] = df_return.apply(set_sample_id, 1)
    sample_exception_indexes = df_mlwh_sample_exceptions.index[
        df_mlwh_sample_exceptions.index.isin(df_return.index)
    ]    
    df_return.loc[sample_exception_indexes, 'derivative_sample_id'] = 'DS_' + df_mlwh_sample_exceptions.loc[sample_exception_indexes, 'sample_id']
    df_return.drop(['sample', 'name'], axis=1, inplace=True)
    
    # Remove any samples that are not in this taxon
    df_return = df_return.loc[~df_return.index.isin(df_mlwh_exceptions_other_taxa.index)]

    # Merge in missing exceptions
    df_return = df_return.append(df_mlwh_missing_exceptions.drop('study_group', 1))
    
    # Remove any samples that don't have an alfresco study
    invalid_codes = ~df_return['alfresco_study_code'].apply(is_valid_alfresco_code)
    if(np.count_nonzero(invalid_codes) > 0):
        print('Removing %d files with invalid alfresco_study_code:' % np.count_nonzero(invalid_codes))
        print(df_return['alfresco_study_code'].loc[invalid_codes].value_counts())
        df_return = df_return.loc[~invalid_codes]

    return(df_return)



In [15]:
df_mlwh_pre_subtrack = create_build_manifest()
print(df_mlwh_pre_subtrack.shape)
print('6901_8_nonhuman#1.bam' in df_mlwh_pre_subtrack.index)
print('6901_8_nonhuman#2.bam' in df_mlwh_pre_subtrack.index)
print('12135_6#65.cram' in df_mlwh_pre_subtrack.index)
print('12135_6#66.cram' in df_mlwh_pre_subtrack.index)


Removing 63 files with invalid alfresco_study_code:
AT-rich amplification                                 53
Test sequencing of high level human contamination.     6
Plasmodium falciparum genome variation 1               4
Name: alfresco_study_code, dtype: int64
(23773, 21)
True
False
True
False


In [16]:
df_mlwh_taxa_exceptions.loc[['6901_8_nonhuman#1.bam', '6901_8_nonhuman#2.bam', '12135_6#65.cram', '12135_6#66.cram'], 'alfresco_study_code']

irods_filename
6901_8_nonhuman#1.bam                                   1052-PF-TRAC-WHITE
6901_8_nonhuman#2.bam             Plasmodium falciparum genome variation 1
12135_6#65.cram                                       1044-PF-KH-FAIRHURST
12135_6#66.cram          Test sequencing of high level human contaminat...
Name: alfresco_study_code, dtype: object

# Conclusion
The missing files were in sequencescape studies that don't have a direct mapping to Alfresco studies. As such we can only attach an Alfresco study if the file was in vw_vrpipe, but these 10 files weren't in vw_vrpipe. My script removes any sample without a valid Alfresco study, and hence these got removed.

Would recommend adding these as manifest exceptions with the explanation "In sequencescape study without corresponding Alfresco study and not in vw_vrpipe".